In [4]:
from rdflib import Graph, Namespace, URIRef

In [ ]:
filename = 'semantic-model.ttl'

In [6]:
g = Graph()
g.parse(filename, format = 'ttl')
BLDG = Namespace("https://BESTESTAir.urn#")
BR = Namespace("https://boptest-rules.urn#")
BRICK = Namespace("https://brickschema.org/schema/Brick#")
g.bind('bldg',BLDG)

In [7]:
pointOfquery = """
PREFIX brick: <https://brickschema.org/schema/Brick#> 
PREFIX boptestrules: <https://boptest-rules.urn#>
CONSTRUCT {
    ?part brick:hasPoint ?point .
}
WHERE {
    ?point boptestrules:pointOf ?type .
    ?parent brick:hasPart ?part .
    ?part a ?type .
}
"""

In [8]:
construct = g.query(pointOfquery)
g = g + construct.graph

In [9]:
print(construct.graph.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<https://BESTESTAir.urn#fcu_fan> brick:hasPoint <https://BESTESTAir.urn#fcu_oveFan>,
        <https://BESTESTAir.urn#fcu_reaPFan> .




In [10]:
# getting rid of all references to the sameAs point, or the weird points it creates.
for keeper, sameAs_pred, remover in g.triples((None, BR['sameAs'], None)):
    print('Keeper: ',keeper,' | Remover: ', remover)
    for remover, hasPoint_pred, fake_points in g.triples((URIRef(remover), BRICK['hasPoint'], None)):
        print('fake points: ', fake_points)
        g.remove((remover, hasPoint_pred, fake_points))
    for remover, hasPart_pred, fake_parts in g.triples((URIRef(remover), BRICK['hasPart'], None)):
        print('fake parts: ', fake_parts)
        g.remove((remover, hasPoint_pred, fake_points))
    g.remove((keeper, sameAs_pred, remover))
    g.remove((remover, hasPart_pred, fake_parts))
    g.remove((remover, hasPoint_pred, fake_points))
    

Keeper:  https://BESTESTAir.urn#con  | Remover:  https://BESTESTAir.urn#con_Thermostat_T
fake points:  https://BESTESTAir.urn#con_Thermostat_T_oveTSetHea
fake points:  https://BESTESTAir.urn#con_Thermostat_T_oveTSetCoo


NameError: name 'hasPart_pred' is not defined

In [ ]:
# Adding thermostat points to zone based on location
query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>

CONSTRUCT {
    ?zon brick:hasPoint ?obj .
}
WHERE {
    ?tstat a brick:Thermostat ;
        brick:hasPoint ?obj ;
        brick:hasLocation|^brick:isLocationOf ?zon .
    ?zon a brick:Zone .
}

"""
res = g.query(query)

In [ ]:
g = g + res.graph

In [ ]:
print(g.serialize())

@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix literal: <https://literal_values.urn#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg:fcu a brick:FCU ;
    brick:feeds bldg:zon ;
    brick:hasPart bldg:fcu_fan ;
    brick:hasPoint bldg:fcu_oveFan,
        bldg:fcu_oveTSup,
        bldg:fcu_reaFloSup,
        bldg:fcu_reaPCoo,
        bldg:fcu_reaPFan,
        bldg:fcu_reaPHea .

bldg:con a brick:Thermostat ;
    brick:hasPoint bldg:con_oveTSetCoo,
        bldg:con_oveTSetHea .

bldg:con_oveTSetCoo_Reference a ref:BOPTestReference ;
    ref:activate literal:con_oveTSetCoo_activate ;
    ref:description literal:description ;
    ref:isWritable true ;
    ref:max literal:max ;
    ref:min literal:min ;

In [ ]:
g.serialize('post-processed-model.ttl', format = 'ttl')

<Graph identifier=N5b98e8afded94db88ef50a0e801e8c84 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Example of Zone Rule

In [12]:
g = Graph()
zone_example = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix boptestrules: <https://boptest-rules.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix literal: <https://literal_values.urn#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

    bldg:ex a brick:Zone_Air_Temperature;

          ref:hasExternalReference bldg:ex_Reference;

          qudt:hasQuantityKind quantitykind:Temperature;

          qudt:hasUnit qudt:K.

    bldg:ex_Reference a ref:BOPTestReference;

          ref:name literal:ex_y;

          ref:description literal:description;

          ref:zone literal:zone;

          ref:unit literal:K;

          ref:isWritable false.
"""
g.parse(data=zone_example, format="turtle")

<Graph identifier=N1137e9b3829e407182b7253480db064d (<class 'rdflib.graph.Graph'>)>

In [ ]:
ofZoneQuery = """

"""